# 视频翻译API服务部署

本笔记本用于在Google Colab上部署视频翻译API服务，并通过ngrok实现公网访问。

## 1. 检查GPU
首先确认是否启用了GPU

In [ ]:
!nvidia-smi

## 2. 安装系统依赖
安装必要的系统包和Python环境

In [ ]:
%%bash
apt update && apt upgrade -y
apt install software-properties-common -y
add-apt-repository ppa:deadsnakes/ppa -y
apt update
apt-get install libxcb-cursor0 -y
apt install python3.10 -y
curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10
update-alternatives --install /usr/bin/python python /usr/local/bin/python3.10 1
apt-get install ffmpeg -y

## 3. 克隆项目并安装依赖
下载项目代码并安装Python依赖包

In [ ]:
!git clone https://github.com/jianchang512/pyvideotrans
%cd pyvideotrans
!pip install -r requirements.txt

# 安装CUDA支持
!pip uninstall -y torch torchaudio
!pip install torch==2.2.0 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu118
!pip install nvidia-cublas-cu11 nvidia-cudnn-cu11

## 4. 配置ngrok
安装并配置ngrok以实现公网访问

**重要**: 请先在[ngrok官网](https://ngrok.com)注册账号并获取Authtoken

In [ ]:
!pip install pyngrok
from pyngrok import ngrok

# ⚠️ 替换为你的 Authtoken
NGROK_AUTH_TOKEN = "你的_NGROK_AUTHTOKEN"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

## 5. 启动API服务
启动API服务并获取公网访问地址。

运行后会显示ngrok生成的公网地址，将其复制并替换到网页客户端的 `api.js` 中的 `baseURL`

In [ ]:
import subprocess
import time
from threading import Thread

def run_api():
    subprocess.run(['python', 'api.py'])

# 在后台启动API服务
api_thread = Thread(target=run_api)
api_thread.daemon = True
api_thread.start()

# 等待API服务启动
time.sleep(5)

# 创建ngrok隧道
public_url = ngrok.connect(9011)
print('='*50)
print(f'API公网访问地址: {public_url}')
print('\n请将此地址替换到网页客户端 js/api.js 中的 baseURL')
print('='*50)

# 保持隧道运行
ngrok_process = ngrok.get_ngrok_process()
try:
    # 显示当前隧道状态
    tunnels = ngrok.get_tunnels()
    print(f'\n当前活动的隧道: {len(tunnels)}')
    for tunnel in tunnels:
        print(f'- {tunnel.public_url} -> {tunnel.config["addr"]}')
    
    print('\n服务已启动，按 Ctrl+C 停止...')
    ngrok_process.proc.wait()
except KeyboardInterrupt:
    print('正在停止服务...')
    ngrok.kill()

## 注意事项

1. 确保已选择 GPU 运行时（Runtime -> Change runtime type -> GPU）
2. 获取到ngrok地址后，需要修改网页客户端的 `js/api.js`：
```javascript
class API {
  constructor(baseURL = "https://xxxx.ngrok.io") { // 替换为实际的ngrok地址
    this.baseURL = baseURL;
  }
  // ...
}
```
3. Colab运行时间有限制，长时间不活动会断开连接
4. 上传到Colab的文件会在会话结束后删除，请注意保存重要文件